# Scikit-Learn IRIS Model using jsonData

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on a kubernetes cluster
 
## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

In [6]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [7]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


Create a seldon config file to deploy the containerized image you just created

In [8]:
%%writefile sklearn_iris_jsondata_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris-jsondata:0.2
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Overwriting sklearn_iris_jsondata_deployment.yaml


In [9]:
!kubectl create -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [10]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" successfully rolled out


### Test by sending prediction calls

jsonData sent as json

In [11]:
import json
res=!curl -s -H "Content-Type: application/json" \
    -d '{"jsonData": {"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}}' \
    "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions"
j=json.loads(res[0])
print(j)
assert(j["data"]["tensor"]["values"][1]>0.0)

{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.0011707012599976396, 0.7876686831680301, 0.21116061557197244]}}, 'meta': {}}


jsonData sent as form-data

In [12]:
res=!curl -s -H 'Content-Type:multipart/form-data' \
    -F jsonData='{"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}' \
    "http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions"
j=json.loads(res[0])
print(j)
assert(j["data"]["tensor"]["values"][1]>0.0)

{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.0011707012599976396, 0.7876686831680301, 0.21116061557197244]}}, 'meta': {}}


In [13]:
!kubectl delete -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted
